In [292]:
class Graph:
    def __init__(self):
        self.adj_dict = {}
        self.node_dict = {}
        self.edge_list = []
        
    def add_edge(self, v, u, pattern):
        if v not in self.adj_dict:
            self.adj_dict[v] = {u:pattern}
        else:
            self.adj_dict[v][u] = pattern
            
        if u not in self.adj_dict:
            self.adj_dict[u] = {v:pattern}
        else:
            self.adj_dict[u][v] = pattern
            
        if v not in self.node_dict:
            self.node_dict[v] = v
            
        if u not in self.node_dict:
            self.node_dict[u] = u
        
        self.edge_list.append([v,u])
        
    def del_edge(self, v, u):
        if u in self.adj_dict[v]:
            del(self.adj_dict[v][u])
            
        if v in self.adj_dict[u]:
            del(self.adj_dict[u][v])
            
    def del_uedge(self, v, u):
        if v in self.adj_dict[u]:
            del(self.adj_dict[u][v])
            
    def edge_l(self):
        return self.edge_list
        
    def adj_d(self, v):
        return self.adj_dict[v]
    
    def vertex(self, v):
        self.adj_list = self.adj_dict[v]
        self.deg = len(self.adj_dict[v])
        self.vertex_num = 1
        
        return self.vertex_num, self.deg, self.adj_list
    
import sys
import time

fileName = 'NCI1'

edges_path = './dataset/' + fileName + '/' + fileName + '.edges' #sys.argv[1]
label_path = './dataset/' + fileName + '/' + fileName + '.node_labels' #sys.argv[2]

label_dict = {}
label_kind = []
nodeList = []
pt_fq = {} # 빈발패턴
label_f = open(label_path, 'r')
# node = 1
while True:
    # line = label_f.readline().replace('\n', '').strip()
    line = label_f.readline().replace('\n', '').strip()
    if not line:
        break
    
    node = line.split(',')[0]
    label = line.split(',')[1]
    # label = line
    label_dict[str(node)] = label
    label_kind.append(label)
    # node += 1
label_f.close

label_kind = list(set(label_kind))

edge_f = open(edges_path, 'r')
gs = Graph()
while True:
    line = edge_f.readline().replace('\n', '').strip()
    if not line:
        break
    
    v = line.split(',')[0]
    u = line.split(',')[1]
    
    if v == u:
        continue
    
    if v not in label_dict:
        label_dict[v] = '0'
        
    if u not in label_dict:
        label_dict[u] = '0'
    
    nodeList.append(v)
    nodeList.append(u)

    pt = label_dict[v] + '/' + label_dict[u]
    
    re_pt = ''.join(reversed(pt))
    
    if pt not in pt_fq and re_pt not in pt_fq:
        pt_fq[pt] = 1
        gs.add_edge(v,u,pt)
        
    elif re_pt in pt_fq:
        pt_fq[re_pt] = pt_fq[re_pt] + 1
        gs.add_edge(v,u,re_pt)
        
    elif pt in pt_fq:
        pt_fq[pt] = pt_fq[pt] + 1
        gs.add_edge(v,u,pt)

edge_f.close

<function TextIOWrapper.close()>

In [293]:
nodeList = list(set(nodeList))
new_pt_fq = {}
new_pt_fq.update(pt_fq)
psize = 1000
pers = 1
p_id = 0
Freq = 0
TF = 0
FR = 0

def Top_nPercent(dict, n):
    if len(dict) == 0:
        return 0
    part_v = len(dict)*n//100

    value = list(sorted(dict.items(), key = lambda item: item[1], reverse = True))[part_v][1]
    return value

def pattern_gen(ptternFrequency, pttern_th, patternSize):
    add_pt = {}
    
    if pttern_th < 0:
        return {}
    
    for pt1 in ptternFrequency:
        if ptternFrequency[pt1] >= pttern_th:
            pt_tgt = pt1.split('/')
            for pt2 in ptternFrequency:
                if ptternFrequency[pt2] >= pttern_th:
                    pt_scr = pt2.split('/')
                    between_len = len(pt_tgt)
                    if between_len == 2:
                        if pt_tgt[0] == pt_scr[0]:
                            pt = pt_scr[1] + '/' + pt1
                        elif pt_tgt[0] == pt_scr[1]:
                            pt = pt_scr[0] + '/' + pt1
                        elif pt_tgt[1] == pt_scr[0]:
                            pt = pt1 + '/' + pt_scr[1]
                        elif pt_tgt[1] == pt_scr[1]:
                            pt = pt1 + '/' + pt_scr[0]
                        else:
                            continue
                    else:
                        between_len = between_len//2

                        if pt_tgt[:-1] == pt_scr[1:]: #앞, 뒤
                            pt = pt_scr[0] + '/' + pt1
                        elif pt_tgt[1:] == pt_scr[:-1]: #뒤, 앞
                            pt = pt1 + '/' + pt_scr[-1]
                        else:
                            continue

                    re_pt = ''.join(reversed(pt))
        
                    if pt not in add_pt and re_pt not in add_pt:
                        
                        if pt1 == pt2:
                            print(pt, pt1, pt2, len(pt_scr))
                            RDP = ptternFrequency[pt1] - ptternFrequency[pt1]//len(pt_scr)
                            if RDP >= pttern_th:
                                add_pt[pt] = RDP
                            
                        else:
                            add_pt[pt] = min(ptternFrequency[pt1], ptternFrequency[pt2])

                    if patternSize + len(add_pt) >= psize:
                        return add_pt
    
    return add_pt

def updatePattern(updateDict, pers):
    patternSize = len(new_pt_fq)
    print(pers, psize)
    if patternSize >= psize:
        update_dict = []
    else:
        update_dict = pattern_gen(updateDict, pers, patternSize)
    if len(update_dict) != 0:
        new_pt_fq.update(update_dict)
        updatePattern(update_dict, pers)

updatePattern(pt_fq, Top_nPercent(pt_fq, pers))
print(sorted(new_pt_fq.items(), key = lambda item: item[1], reverse = True)[:100])
print(len(new_pt_fq))

148304 1000
3/3/3 3/3 3/3 2
[('3/3', 148304), ('3/1', 43368), ('3/2', 37234), ('3/4', 3656), ('2/1', 2024), ('3/5', 1892), ('3/7', 1420), ('2/2', 1404), ('1/4', 1298), ('1/6', 458), ('3/6', 356), ('2/4', 346), ('1/1', 336), ('3/9', 254), ('3/14', 213), ('14/3', 213), ('3/22', 136), ('1/5', 100), ('2/6', 70), ('4/4', 56), ('2/5', 44), ('1/22', 40), ('3/21', 38), ('21/3', 38), ('27/1', 22), ('1/27', 22), ('7/2', 20), ('5/10', 16), ('10/5', 16), ('1/7', 16), ('1/26', 15), ('3/26', 15), ('26/1', 15), ('26/3', 15), ('5/9', 14), ('6/4', 12), ('8/1', 12), ('7/7', 12), ('27/3', 11), ('3/27', 11), ('7/4', 10), ('3/19', 10), ('19/3', 10), ('11/3', 10), ('10/3', 10), ('3/10', 10), ('14/2', 7), ('2/14', 7), ('5/5', 6), ('4/26', 5), ('26/4', 5), ('3/25', 4), ('25/3', 4), ('2/17', 4), ('17/2', 4), ('8/3', 4), ('3/17', 3), ('17/3', 3), ('3/30', 3), ('30/3', 3), ('31/3', 3), ('3/31', 3), ('1/37', 2), ('37/1', 2), ('7/29', 2), ('29/7', 2), ('3/28', 2), ('28/3', 2), ('2/21', 2), ('21/2', 2), ('31/1', 2)

In [315]:
batchsize = 3
DxMtrix = [[0 for col in range(batchsize+3)] for row in range(len(new_pt_fq))]
new_pt = list(sorted(new_pt_fq.items(), key = lambda item: item[1], reverse = True))



In [317]:
for line in range(len(DxMtrix)):
    DxMtrix[line][0] = new_pt[line][0]
    DxMtrix[line][1] = new_pt[line][1]
    DxMtrix[line][4] = sum(DxMtrix[line][1:4])
    
    
print(DxMtrix)

[['3/3', 148304, 0, 0, 148304, 0], ['3/1', 43368, 0, 0, 43368, 0], ['3/2', 37234, 0, 0, 37234, 0], ['3/4', 3656, 0, 0, 3656, 0], ['2/1', 2024, 0, 0, 2024, 0], ['3/5', 1892, 0, 0, 1892, 0], ['3/7', 1420, 0, 0, 1420, 0], ['2/2', 1404, 0, 0, 1404, 0], ['1/4', 1298, 0, 0, 1298, 0], ['1/6', 458, 0, 0, 458, 0], ['3/6', 356, 0, 0, 356, 0], ['2/4', 346, 0, 0, 346, 0], ['1/1', 336, 0, 0, 336, 0], ['3/9', 254, 0, 0, 254, 0], ['3/14', 213, 0, 0, 213, 0], ['14/3', 213, 0, 0, 213, 0], ['3/22', 136, 0, 0, 136, 0], ['1/5', 100, 0, 0, 100, 0], ['2/6', 70, 0, 0, 70, 0], ['4/4', 56, 0, 0, 56, 0], ['2/5', 44, 0, 0, 44, 0], ['1/22', 40, 0, 0, 40, 0], ['3/21', 38, 0, 0, 38, 0], ['21/3', 38, 0, 0, 38, 0], ['27/1', 22, 0, 0, 22, 0], ['1/27', 22, 0, 0, 22, 0], ['7/2', 20, 0, 0, 20, 0], ['5/10', 16, 0, 0, 16, 0], ['10/5', 16, 0, 0, 16, 0], ['1/7', 16, 0, 0, 16, 0], ['1/26', 15, 0, 0, 15, 0], ['3/26', 15, 0, 0, 15, 0], ['26/1', 15, 0, 0, 15, 0], ['26/3', 15, 0, 0, 15, 0], ['5/9', 14, 0, 0, 14, 0], ['6/4', 12, 0

In [ ]:
f = open('./Result/' + fileName + '_' + str(len(new_pt_fq)) + '_' + str(pers) + '.node_labels', 'w')

for e in dxMatrix:
    str_e = e[0] + ',' + e[1] + '\n'
    f.write(str_e)
        
f.close()

In [294]:
allEdgeDict = {}

def neighborScore(v, u):
    deg_max = gs.vertex(v)[1]
    same_deg = len((gs.vertex(v)[2].keys() & gs.vertex(u)[2].keys()))
    deg_ratio = gs.vertex(u)[1]/gs.vertex(v)[1]
            
    score = (same_deg/deg_max) + deg_ratio
    
    return score

def pattern_Marge(v):
    score = {}
    neighbor_v = gs.vertex(v)[2]
    
    for u in neighbor_v:
        if u in allEdgeDict:
            continue
        re_pt = ''.join(reversed(neighbor_v[u]))
        if neighbor_v[u] in new_pt_fq.keys() or re_pt in new_pt_fq.keys():
            score[u] = neighborScore(v, u)
    
    if score:
        su = list(sorted(score.items(), key = lambda item: item[1], reverse = True))[0]
        gs.del_edge(v, su[0])
        MargeNode = v + '/' + su[0]
        patternExtension = label_dict[v] + '/' + label_dict[su[0]]
        return MargeNode, patternExtension
    else:
        return 0, 0
    
def pattern_Marge2(MargeNode, patternExtension):
    MargeNodeList = MargeNode.split('/')
    frontV = MargeNodeList[0]
    backV = MargeNodeList[-1]
    frontMax = [0,0]
    backMax = [0,0]
    union_list1 = {}
    union_list2 = {}
    frontScore = {}
    backScore = {}

    for front in gs.vertex(frontV)[2].keys():
        if front in allEdgeDict or front in MargeNodeList:
            continue
        union_list1[front] = label_dict[front] + '/' + patternExtension
        
    for back in gs.vertex(backV)[2].keys():
        if back in allEdgeDict or back in MargeNodeList:
            continue
        union_list2[back] = patternExtension + '/' + label_dict[back]
            
    for union in union_list1.keys():
        re_pt = ''.join(reversed(union_list1[union]))
        if union_list1[union] in new_pt_fq.keys() or re_pt in new_pt_fq.keys():
            frontScore[union] = neighborScore(frontV, union)
            
    for union in union_list2.keys():
        re_pt = ''.join(reversed(union_list2[union]))
        if union_list2[union] in new_pt_fq.keys() or re_pt in new_pt_fq.keys():
            backScore[union] = neighborScore(backV, union)

    if frontScore:
        frontMax = list(sorted(frontScore.items(), key = lambda item: item[1], reverse = True))[0]

    if backScore:
        backMax = list(sorted(backScore.items(), key = lambda item: item[1], reverse = True))[0]
    
    if frontMax[1] >= backMax[1] and frontMax[1] != 0:
        gs.del_edge(frontV, frontMax[0])
        MargeNode = frontMax[0] + '/' + MargeNode
        new_pt = union_list1[frontMax[0]]
        
    elif frontMax[1] < backMax[1] and backMax[1] != 0:
        gs.del_edge(backV, backMax[0])
        MargeNode = MargeNode + '/' + backMax[0]
        new_pt = union_list2[backMax[0]]
        
    else:
        return MargeNode, patternExtension, 0
        
    return MargeNode, new_pt, 1
    
pattern_dict = {}
edge_dict = {}
s1 = time.time()
pattern_count = {}
while(True):
    edgeList = []
    node_size = len(nodeList)
    
    if node_size <= 0:
        e1 = time.time()
        d1 = e1-s1
        print(node_size, d1)
        break
    
    node = nodeList[0]
    newV, pE = pattern_Marge(node)
    
    if newV == 0:
        nodeList.remove(node)
        continue
    
    boolz = 1
    while (boolz > 0):
        newV, pE, boolz = pattern_Marge2(newV, pE)
    
    del_list = newV.split('/')
    for del_one in del_list:
        nodeList.remove(del_one)
        edge_list = list(gs.vertex(del_one)[2].keys())                
        edgeList.extend(edge_list)
        allEdgeDict[del_one] = newV
    
    edge_dict[newV] = edgeList
    
    if pE not in pattern_dict:
        pattern_dict[pE] = [newV]
        pattern_count[pE] = len(edgeList)
    else:
        pattern_dict[pE].append(newV)
        pattern_count[pE] = pattern_count[pE] + len(edgeList)

    


0 35.74834156036377


In [295]:
patternList = list(sorted(pattern_count.items(), key = lambda item: item[1], reverse = True))
patternRank = []
for pl in patternList:
    patternRank.append(pl[0])

In [296]:
P_id_dict = {}
c0 = 0
for PT in patternRank:
    P0 = "P" + str(c0)
    P_id_dict[PT] = P0
    c0 = c0 + 1

GP_id_dict = {}
for i in pattern_dict:
    P1 = P_id_dict[i]
    c1 = 0
    for j in pattern_dict[i]:
        P2 = P1 + '-' + str(c1)
        GP_id_dict[j] = P2
        c1 = c1 + 1
        
        
GP_id_dict2 = {}
for i in pattern_dict:
    P1 = P_id_dict[i]
    c1 = 0
    for j in pattern_dict[i]:
        P2 = P1 + '-' + str(c1)
        GP_id_dict2[P2] = P1
        c1 = c1 + 1
        

In [297]:
edge_dict2 = {}

for pd in edge_dict:
    del_list = []
    for pd2 in edge_dict[pd]:
        if pd2 in allEdgeDict:
            del_list.append(pd2)
            edge_dict[pd].append(allEdgeDict[pd2])
            
    for del_one in del_list:
        edge_dict[pd].remove(del_one)
    edge_dict[pd] = list(set(edge_dict[pd]))

    if pd in GP_id_dict:
        n_pd = GP_id_dict[pd]
    else:
        n_pd = pd

    for pd2 in edge_dict[pd]:
        if pd2 in GP_id_dict:
            n_pd2 = GP_id_dict[pd2]
        else:
            n_pd2 = pd2
            
        if n_pd not in edge_dict2:
            edge_dict2[n_pd] = [n_pd2]
        else:
            edge_dict2[n_pd].append(n_pd2)
            


In [298]:
pEdgeList = []
pNodeDict = {}
for pd in edge_dict2:
    if pd not in pNodeDict:
        if pd in label_dict:
            pNodeDict[pd] = label_dict[pd]
        elif pd in GP_id_dict2:
            pNodeDict[pd] = GP_id_dict2[pd]

    for pd2 in edge_dict2[pd]:
        if pd == pd2:
            continue
        if [pd, pd2] not in pEdgeList and [pd2, pd] not in pEdgeList:
            pEdgeList.append([pd, pd2])
        if pd2 not in pNodeDict:
            if pd2 in label_dict:
                pNodeDict[pd2] = label_dict[pd2]
            elif pd2 in GP_id_dict2:
                pNodeDict[pd2] = GP_id_dict2[pd2]


In [299]:
print(len(pNodeDict))
print(len(pEdgeList))

61995
70817


In [300]:
Ge = pEdgeList

f = open('./Result/' + fileName + '_' + str(len(new_pt_fq)) + '_' + str(pers) + '.edges', 'w')

for e in Ge:
    str_e = e[0] + ',' + e[1] + '\n'
    f.write(str_e)
        
f.close()



In [301]:
P_id_list = list(sorted(P_id_dict.items(), key = lambda item: item[1], reverse = True))
Gp = P_id_list
f = open('./Result/' + fileName + '_' + str(len(new_pt_fq)) + '_' + str(pers) + '.pattern', 'w')

for e in Gp:
    str_e = e[1] + ',' + e[0] + '\n'
    f.write(str_e)
        
f.close()

In [302]:
Gl = list(sorted(pNodeDict.items()))
f = open('./Result/' + fileName + '_' + str(len(new_pt_fq)) + '_' + str(pers) + '.node_labels', 'w')

for e in Gl:
    str_e = e[0] + ',' + e[1] + '\n'
    f.write(str_e)
        
f.close()

In [ ]:
import networkx as nx
from pyvis.network import Network
G = nx.Graph()

for v, u in pEdgeList:
    G.add_edge(v,u)
    
graph_name = fileName + '_pattern.html'
net = Network(bgcolor="#222222", font_color="white", filter_menu=True, select_menu=True)
net.from_nx(G)

net.show_buttons(filter_=['physics'])
net.show(graph_name)